In [11]:
import pandas as pd
import os
import openpyxl
import numpy as np
import re
from datetime import datetime

In [15]:
class FinancialReport():
    def __init__(self,file):
        self.file = file
        self.readFile()
        self.findBalanceSheet()
    def readFile(self):
        path = os.path.join("financial statements",self.file)
        self.statements = pd.read_excel(path,sheet_name=None)
    def getSheetNames(self):
        return self.statements.keys()
    def findBalanceSheet(self):
        balanceSheetName = [name for name in self.getSheetNames() if "BALANCE" in name][0]
        self.balance = self.statements[balanceSheetName]
        self.balance.columns = ["label"]+list(self.balance.columns[1:])
    def getBalanceSheet(self):
        return self.balance
    

In [37]:
class Balance():
    def __init__(self):
        self.finalBalance = pd.DataFrame(columns=["title","title_order","label"])
        self.setFinancialStatements()
        self.addDataToBalanceSheet()
#         self.rearrangeFinalBalanceSheet()
#         self.setQuarters()
        
    def getFiles(self):
        files = os.listdir("financial statements")
        files.remove(".DS_Store")
        files = [file for file in files if file[0] != "~"]
        return files

    def setFinancialStatements(self):
        self.files = self.getFiles()
        self.FinancialReports = [FinancialReport(file) for file in self.files]
        
    def addDataToBalanceSheet(self):
        
        for FinancialStatements in self.FinancialReports:
            balance = FinancialStatements.getBalanceSheet().copy()
            
            cols = ["label"]+[col for col in balance.columns[1:] if col not in self.finalBalance.columns] ## remove columns of data already in main balance sheet
            balance = balance[cols]
            
            balance = self.parseStatement(balance)
            
            self.finalBalance = pd.merge(self.finalBalance, balance, how="outer",on=["label","title"],suffixes=('', '_x'))
            
            self.finalBalance.title_order = [self.merge_number(self.finalBalance.title_order[i],self.finalBalance.title_order_x[i]) for i in range(len(self.finalBalance))] #merge the number for the titles. use greatest num bc bigger num means there is section before it
            self.finalBalance = self.finalBalance.drop(["title_order_x"],axis=1)

    def rearrangeFinalBalanceSheet(self):
        self.finalBalance = self.finalBalance.replace(" ",np.nan)

        # put totals at the bottom of each section
        totals = self.finalBalance.loc[self.finalBalance.label.str.startswith("total")].reset_index(drop=True) 
        self.finalBalance = self.finalBalance.loc[~self.finalBalance.label.str.startswith("total")].reset_index(drop=True)
        self.finalBalance = pd.concat([self.finalBalance,totals]) 
        self.finalBalance = self.finalBalance.reset_index(drop=True).reset_index()
        self.finalBalance = self.finalBalance.sort_values(["title_order","index"]) # order by title and then location
        self.finalBalance = self.finalBalance.drop(["index","title_order"],axis=1)

    def parseStatement(self,sheet):
        num = 0
        index = []
        title = False
        for label in sheet.label:
            if ":" in label:
                title = re.sub("\([\w\W]+\)","",label[:-1]).replace("  "," ")
                num += 1
                continue
            if title:
                new_label = label.lower()
                new_label = new_label.replace("gain","loss")
                new_label = re.sub("\([\w\W]+\)","",new_label).replace("  "," ").strip()
                new_label = re.sub(" \$|\s*\d+\,*", "", new_label) 
                new_label = self.clean_up_string(new_label)
                
                print([title,num,label,new_label])
                index.append([title,num,label,new_label])
            
            if "total" == new_label[:5]:
                title = np.nan
                num += 1

        sheet = sheet.loc[sheet.label.isin(np.array(index)[:,2])].copy()
        sheet.loc[:,["title","title_order","label"]] = list(np.array(index)[:,[0,1,3]])
        sheet = sheet.set_index(["title","title_order","label"]).reset_index()
        return sheet
    
    def clean_up_string(self,string):
        string = string.strip()
        space_positions = [x.start() for x in re.finditer('\ ',  string)]
        for i in reversed(space_positions):
            if string[i+1:] in ["of", "and"]:
                string = string[:i]
        return string
    
    def merge_number(self,a,b):
        if pd.isnull(a):
            return b
        if pd.isnull(b):
            return a
        return max(a,b)
    def setQuarters(self):
        dates = list(self.finalBalance.columns[2:])
        dates.sort(key=lambda date: datetime.strptime(date.replace(".",""), '%b %d, %Y'))
        self.finalBalance = self.finalBalance[["title","label"]+dates]
        self.finalBalance.columns = ["title","label"]+[self.parseQuarter(date) for date in self.finalBalance.columns[2:]]
    def parseQuarter(self,date):
        mo = date.split(" ")[0].strip(".")
        year = date.split(", ")[-1]
        if mo in ["May"]:
            return "Q1 "+str(int(year)+1)
        
        if mo in ["Jul","Aug"]:
            return "Q2 "+str(int(year)+1)
        
        if mo in ["Oct","Nov"]:
            return "Q3 "+str(int(year)+1)
        
        if mo in ["Jan","Feb"]:
            return "Q4 "+str(int(year))
        
        

In [38]:
balance = Balance()
balance.finalBalance

['Current assets', 1, 'Cash and cash equivalents', 'cash and cash equivalents']
['Current assets', 1, 'Short-term investments', 'short-term investments']
['Current assets', 1, 'Merchandise inventory', 'merchandise inventory']
['Current assets', 1, 'Other current assets', 'other current assets']
['Current assets', 1, 'Total current assets', 'total current assets']
[nan, 2, 'Property and equipment, net of accumulated depreciation', 'property and equipment, net of accumulated depreciation']
[nan, 2, 'Operating Lease, Right-of-Use Asset', 'operating lease, right-of-use asset']
[nan, 2, 'Other long-term assets', 'other long-term assets']
[nan, 2, 'Total assets', 'total assets']
['Current liabilities', 4, 'Accounts payable', 'accounts payable']
['Current liabilities', 4, 'Accrued expenses and other current liabilities', 'accrued expenses and other current liabilities']
['Current liabilities', 4, 'Operating Lease, Liability, Current', 'operating lease, liability, current']
['Current liabiliti

,title_order,title,label,"Oct. 30, 2021","Jan. 30, 2021","Oct. 31, 2020","Feb. 01, 2020","Nov. 02, 2019","May 04, 2019","Feb. 02, 2019","May 05, 2018","Jul. 31, 2021","Aug. 01, 2020","Aug. 03, 2019","Feb. 03, 2018","May 01, 2021","May 02, 2020","Aug. 04, 2018","Jan. 29, 2022","Nov. 03, 2018"
0,1,Current assets,cash and cash equivalents,801,1988,2471,1364,788,941,1081,1210,2375,2188,1177,1783.0,2066,1028,1322,877.00,958
1,1,Current assets,short-term investments,275,410,178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.00,NaN
2,1,Current assets,merchandise inventory,2721,2451,2747,2156,2720,2242,2131,2035,2281,2242,2326,1997.0,2370,2217,2202,3018.00,2668
3,1,Current assets,other current assets,1410,1159,966,706,770,757,751,778,1201,882,770,788.0,1091,920,780,1270.00,792
4,1,Current assets,total current assets,5207,6008,6362,4516,4572,4212,4251,4187,6194,5337,4567,4568.0,6002,4216,4590,5165.00,4714
5,2,nan,"property and equipment, net of accumulated dep...",2924,2841,2846,3122,3225,3129,2912,2791,2897,2895,3141,2805.0,2839,2945,2832,3037.00,2887
6,2,nan,"operating lease, right-of-use asset",3788,4217,4460,5402,5796,5732,0,0,3975,4689,5807,NaN,4060,4851,0,NaN,0
7,2,nan,other long-term assets,861,703,705,639,525,547,886,607,693,795,528,616.0,703,698,588,884.00,572
8,2,nan,total assets,12780,13769,14373,13679,14118,13620,8049,7585,13759,13716,14043,7989.0,13604,12710,8010,12761.00,8173
9,4,Current liabilities,accounts payable,1630,1743,2284,1174,1241,994,1126,1072,1583,1629,1246,1181.0,1530,971,1297,1951.00,1299


In [24]:
balance.finalBalance.to_csv("finalBalance2.csv")

In [79]:
'Oct. 30, 2021'.split(", ")[-1]

'2021'